In [14]:
directory = 'C:/Users/Evan/Documents/PhD/DCIM/100MSDCF'

flightLogDirectoryInput = input('Enter the flight log path: ')
flightLogDirectory = flightLogDirectoryInput.replace("\\", "/")
#flightLogDirectory = 'D:/ATEC/120717_sample_F_WK/sony/flight_logs/FLY202_20170712_WK_F1.csv'

import os
from shutil import copy

with open(directory+'/jpgImageTimes.csv') as jpgFile:
    jpgFileLines = jpgFile.read().split("\n")
    jpgjpgImageNames = [(jpgFileLines[x]).split(',')[0] for x in range(0, len(jpgFileLines)-1)]
    jpgImageTimes = [(jpgFileLines[x]).split(',')[1] for x in range(0, len(jpgFileLines)-1)]
       
def howManySecs(clockFace):
    secs = int(clockFace[-2:]) + 60*int(clockFace[3:5]) + 60*60*int(clockFace[:2])
    return secs

with open(flightLogDirectory) as flightFile:      
    flightFileLines = flightFile.read().split("\n")

    camShotLine = []
    word = 'SUCCESSED SIMPLE_SHOT'

    for i,line in enumerate(flightFileLines):
        if word in line: # or word in line.split() to search for full words
            camShotLine.append(i)

    missionRangeLine = []
    startWord = '[L-MIS]WP Mission: received idle vel cmd 3'
    endWord = 'turn mode. end of trace. quit'

    for i,line in enumerate(flightFileLines):
        if startWord in line: # or word in line.split() to search for full words
            missionRangeLine.append(i)
            missStartLine = i
            print("Word \"{}\" found in line {}".format(startWord, i))

    for j,line in enumerate(flightFileLines):
        if endWord in line: # or word in line.split() to search for full words
            missionRangeLine.append(j)
            missEndLine = j
            print("Word \"{}\" found in line {}".format(endWord, j))

    potMatch = len(camShotLine)
    potMatchLines = [flightFileLines[x] for x in camShotLine]

    dateWord = '|#SiF%|2'

    times = []

    for y in range(potMatch):
        charDateWord = potMatchLines[y][(potMatchLines[y].find(dateWord)+7):((potMatchLines[y].find(dateWord)+15))]
        if potMatchLines[y].find(charDateWord+".0")>1:
            times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".0")+9):((potMatchLines[y].find(charDateWord+".0")+17))])
        elif potMatchLines[y].find(charDateWord+".1")>1:  
            times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".1")+9):((potMatchLines[y].find(charDateWord+".1")+17))])
        elif potMatchLines[y].find(charDateWord+".2")>1:
            times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".2")+9):((potMatchLines[y].find(charDateWord+".2")+17))])

    flightTimes = [(flightFileLines[x]).split(',')[1] for x in camShotLine]

    missStartFlightTime = (flightFileLines[missStartLine]).split(',')[1]
    missFinishFlightTime = (flightFileLines[missEndLine]).split(',')[1]
    flightTimeDiffs = [float(missStartFlightTime)-float(flightTimes[0]), float(missFinishFlightTime)-float(flightTimes[0])]
    realFlightTimes = [0,0]
    for j in range(2):
        timeHolder = (times[0]).split(':')
        for i in range(3):
            timeHolder[i] = float(timeHolder[i])
        timeHolder[2] = timeHolder[2] + flightTimeDiffs[j]
        while timeHolder[2] >= 60:
            timeHolder[2] = timeHolder[2] - 60
            timeHolder[1] = timeHolder[1] + 1
            while timeHolder[1] >= 60:
                timeHolder[1] = timeHolder[1] - 60
                timeHolder[0] = timeHolder[0] + 1
        while timeHolder[2] <= 0:
            timeHolder[2] = timeHolder[2] + 60
            timeHolder[1] = timeHolder[1] - 1
            while timeHolder[1] <= 0:
                timeHolder[1] = timeHolder[1] + 60
                timeholder[0] = timeHolder[0] - 1
        realFlightTimes[j] = timeHolder

    offsetTimes = [(flightFileLines[x]).split(',')[1] for x in range(1, len(flightFileLines)-1)]
    longitudes = [(flightFileLines[x]).split(',')[3] for x in range(1, len(flightFileLines)-1)]
    latitudes = [(flightFileLines[x]).split(',')[4] for x in range(1, len(flightFileLines)-1)]
    altitudes = [(flightFileLines[x]).split(',')[10] for x in range(1, len(flightFileLines)-1)]

    startSecond = 60*60*realFlightTimes[0][0] + 60*realFlightTimes[0][1] + realFlightTimes[0][2]
    endSecond = 60*60*realFlightTimes[1][0] + 60*realFlightTimes[1][1] + realFlightTimes[1][2]
    zerothRealTime = startSecond - (float(offsetTimes[missStartLine-1]) - float(offsetTimes[0]))

    with open(directory+'jpgFlightFileScript2.csv', 'w') as outputFlightFile:
        firstRow = "Mission Start Line" + "," + str(missStartLine) + "," + "Mission End Line" + "," + str(missEndLine) + "\n"
        outputFlightFile.write(firstRow)
        secondRow = "Seconds" + "," + "Longitude" + "," + "Latitude" + "," + "Altitude" + "\n"
        outputFlightFile.write(secondRow)
        for i in range(0, len(flightFileLines)-2):  #I took out the +/- 20 buffer here, maybe erroneously
            timeInSeconds = float(offsetTimes[i]) - float(offsetTimes[0]) + zerothRealTime
            row =  str(timeInSeconds) + "," + longitudes[i] + "," + latitudes[i] + "," + altitudes[i] + "\n"
            outputFlightFile.write(row)

    print("All done :)")
    
for offSec in range(0, 30):
    
    newpath1 = directory + 'MissionData'+str(offSec)+'jpg/'  
    if not os.path.exists(newpath1):
        os.makedirs(newpath1)

    jpgImageSeconds = []
    for j in range(1, len(jpgFileLines)-1):
        jpgImageSeconds.append(howManySecs(jpgImageTimes[j])-3600-offSec) #The -3600 is to account for BST --> UTC

    with open(directory+str(offSec)+'jpgImageTimesFirstScript.csv', 'w') as jpgFile:
        firstRow = jpgjpgImageNames[0] + "," + jpgImageTimes[0] + "," + "Seconds" + "\n"
        jpgFile.write(firstRow)
        secondRow = jpgjpgImageNames[1] + "," + jpgImageTimes[1] + "," + str(jpgImageSeconds[0]) + "\n"
        jpgFile.write(secondRow)
        for i in range(1, len(jpgFileLines)-2):
            row = jpgjpgImageNames[i+1] + "," + jpgImageTimes[i+1] +  "," + str(jpgImageSeconds[i]) + "\n"
            jpgFile.write(row)     
    
    with open(directory+'jpgFlightFileScript2.csv', 'r') as inputFlightFile:
        missionLines = inputFlightFile.read().split("\n")

    with open(directory+str(offSec)+'jpgImageTimesFirstScript.csv', 'r') as inputjpgFile:
        jpgLines = inputjpgFile.read().split("\n")

    missionSeconds = [float((missionLines[x]).split(',')[0]) for x in range(2, len(missionLines)-1)]
    intMissionSeconds = [int(missionSeconds[x]) for x in range(len(missionSeconds))]
    longitudes = [(missionLines[x]).split(',')[1] for x in range(2, len(missionLines)-1)]
    latitudes = [(missionLines[x]).split(',')[2] for x in range(1, len(missionLines)-1)]
    altitudes = [(missionLines[x]).split(',')[3] for x in range(1, len(missionLines)-1)]
    jpgImageNames = [(jpgLines[x]).split(',')[0] for x in range(1, len(jpgLines)-1)]
    jpgSeconds = [int((jpgLines[x]).split(',')[2]) for x in range(1, len(jpgLines)-1)]

    cnt = 0
    jpgSecondsLineNum = []

    #will find 46 in 746 btw so be carfeul!!!!!
    for sec in jpgSeconds:
        cnt = 0
        while cnt==0:
            try:
                jpgSecondsLineNum.append(intMissionSeconds.index(sec))
                break
            except ValueError:
                #print(sec, "doesn't match any of our records, sorry")
                cnt = cnt + 1
                jpgSecondsLineNum.append(intMissionSeconds.append(0))            

    with open(directory+str(offSec)+'jpgImageFinalMisisonDataScript3.csv', 'w') as jpgFinalDataFile:
        rowCounter = 0
        firstRow = "File Name" + "," + "Longitude" + "," + "Latitude" + "," + "Altitude" + "\n"
        jpgFinalDataFile.write(firstRow)
        for i in range(len(jpgSeconds)):
            lineNum = jpgSecondsLineNum[i]
            if type(lineNum) == int or type(lineNum) == float:
                if lineNum >= missStartLine and lineNum <= missEndLine:
                    #if rowCounter >= 0 and rowCounter <= 55:
                    row = jpgImageNames[i] + "," + longitudes[lineNum] + "," + latitudes[lineNum] + "," + altitudes[lineNum] + "\n"
                    jpgFinalDataFile.write(row)
                    rowCounter = rowCounter + 1
                else:
                    rowCounter = rowCounter + 1

    print("All done with that one :)")
    
    jpgHolder = []
    jpgCounter = 0
    for file in os.listdir(directory):
        if file.endswith(".jpg"):
            jpgHolder.append(os.path.join(directory, file))
            jpgCounter = jpgCounter + 1

    with open(directory+str(offSec)+'jpgImageFinalMisisonDataScript3.csv', 'r') as jpgFinalDataFile:
        jpgLines2 = jpgFinalDataFile.read().split("\n")
        jpgImageNames = [(jpgLines2[x]).split(',')[0] for x in range(1, len(jpgLines2)-1)]
        print(jpgImageNames)
        tempImage = (jpgImageNames[0]).split('\\')
        fileLocal = len(tempImage)
        jpgImageFileNames = [(jpgImageNames[x]).split('\\')[fileLocal-1] for x in range(len(jpgImageNames))]
        for i in range(len(jpgImageNames)):
            #os.rename(jpgImageNames[i],  newpath1 + jpgImageFileNames[i])
            copy(jpgImageNames[i], newpath1 + jpgImageFileNames[i])

    print("All done :)")


Enter the flight log path: C:\Users\Evan\Documents\PhD\flight_logs\FLY380.csv
Word "[L-MIS]WP Mission: received idle vel cmd 3" found in line 5963
Word "turn mode. end of trace. quit" found in line 8052
All done :)
All done with that one :)
['C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06487.JPG']
All done :)
All done with that one :)
['C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06487.JPG']
All done :)
All done with that one :)
['C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06487.JPG']
All done :)
All done with that one :)
['C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06487.JPG']
All done :)
All done with that one :)
['C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06487.JPG']
All done :)
All done with that one :)
['C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06487.JPG', 'C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06488.JPG']
All done :)
All done with that one :)
['C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06487.JPG', 'C:/Users/Evan/Documents/PhD/DCIM/100MSDCF\\DSC06488

In [ ]:
C:\Users\Evan\Documents\PhD\flight_logs\FLY380.csv